# Notebook #2: Limpieza: limpieza y transformación de los datos

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../src')
import funciones as f
import soporte_prep as sp

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

In [2]:
df_infracciones_2 = pd.read_excel("../../Datos/infracciones/Infracciones de Tránsito, DIGESETT, 2018 - 2024.xlsx")
df_infracciones_1 = pd.read_excel("../../Datos/infracciones/Infracciones de Tránsito, DIGESETT, 2010 - 2017.xlsx")

In [3]:
df_infracciones_1.columns = ['infracciones', 'total', 'anio']
df_infracciones_2.columns = df_infracciones_2.columns.str.lower() #pongo los nombres en minúsculas
df_infracciones_2.rename(columns={'año':'anio'}, inplace=True)

In [4]:
df_infracciones_2['total'] = df_infracciones_2.iloc[:,1:13].sum(axis=1) # creo la columna de total

In [5]:
df_infracciones_2.reset_index(drop=True, inplace=True)

In [6]:
df_infracciones_2.columns = df_infracciones_2.columns.str.strip() # quito posibles espacios no vistos

In [7]:
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].replace({
    'TRANSITAR EN VIA CONTRARIA': 'VIA CONTRARIA'
})

In [8]:
# Calculo el porcentaje dentro de cada año usando transform
df_infracciones_1['porcentaje'] = df_infracciones_1.groupby('anio')['total'].transform(lambda x: (x / x.sum()) * 100)

In [9]:
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].str.strip()
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].str.lower()

In [10]:
df_infracciones_1= df_infracciones_1[['anio', 'infracciones', 'total', 'porcentaje']]

In [11]:
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].replace({
    'estacionarse en lugares prohibido': 'estacionarse en lugares prohibidos'})

In [12]:
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].replace({
    'licencia vencida': 'conducir sin licencia y/o vencida', 
    'conducir sin licencia': 'conducir sin licencia y/o vencida'})

In [13]:
# calculo el porcentaje dentro de cada año usando transform
df_infracciones_2['porcentaje'] = df_infracciones_2.groupby('anio')['total'].transform(lambda x: (x / x.sum()) * 100)

# Filtro para quedarme con los valores totales a nivel anual
df_infracciones_2 = df_infracciones_2[['anio', 'infracciones', 'total', 'porcentaje']]

In [14]:
# Quito posibles elementos que no me permitan trabajar la columna correctamente
df_infracciones_2['infracciones'] = df_infracciones_2['infracciones'].str.replace(
    r'^\d+\.?\s*|\(.*?\)', '', regex=True
).str.strip()

In [15]:
df_infracciones_2['porcentaje'] = df_infracciones_2['porcentaje'].round(2)
df_infracciones_2['infracciones']=df_infracciones_2['infracciones'].str.replace('-', '')
df_infracciones_2['infracciones'] = df_infracciones_2['infracciones'].str.lower()

In [16]:
mapeo_multiple = {
    # Estacionarse en lugares prohibidos
    'estacionarse en lugar prohibido': 'estacionarse en lugares prohibidos',
    'estacionarse en lugares prohibido': 'estacionarse en lugares prohibidos',
    'estacionarse en lugares prohibidos': 'estacionarse en lugares prohibidos',
    
    # Obstruir el tránsito
    'parar en la intersecciones / obstruir el tránsito': 'obstruir el transito',
    'obstruir el tránsito': 'obstruir el transito',
    
    # Conducir sin licencia y/o vencida
    'transitar sin licencia y/o vencida': 'conducir sin licencia y/o vencida',
    'transitar sin permiso': 'conducir sin licencia y/o vencida',
    'conducir sin licencia': 'conducir sin licencia y/o vencida',
    'licencia vencida': 'conducir sin licencia y/o vencida',
    'transitar sin licencia o vencida': 'conducir sin licencia y/o vencida',
    
    # Vía contraria
    'transitar en vía contraria': 'via contraria',
    'vía contraria': 'via contraria',
    
    # Hablar por celular
    'uso de celular o dispositivos electrónicos': 'hablar por celular',
    'uso del celular o dispositivos electrónicos  art. 221)': 'hablar por celular',
    
    # Transitar sin cinturón
    'transitar sin cinturón': 'transitar sin cinturon',
    'no uso del cinturón de seguridad': 'transitar sin cinturon',
    
    # Transitar sin casco
    'transitar sin casco protector': 'transitar sin casco',
    'no uso del casco protector': 'transitar sin casco',
    
    # Violar la luz roja
    'violar luz roja': 'violar la luz roja',
    'respeto a las señales del semáforo': 'violar la luz roja',
    
    # Giro prohibido
    'realizar giro prohibido': 'giro prohibido'
}

In [17]:
# Quito espacios y mapeo la columna para reemplazar esos valores a su categoría correspondiente
df_infracciones_2['infracciones'] = df_infracciones_2['infracciones'].str.strip()
df_infracciones_2 = f.reemplazar_valores(
    dataframe=df_infracciones_2, 
    columna='infracciones', 
    mapeo=mapeo_multiple
)

In [18]:
df_infracciones_2_dirt = df_infracciones_2.copy()

In [19]:
infracciones_validas = ['violar la luz roja', 'transitar sin casco',
       'transitar sin cinturon', 'hablar por celular', 'via contraria',
       'conducir sin licencia y/o vencida',
       'obstruir el transito', 'estacionarse en lugares prohibidos',
       'giro prohibido']

# Convertir todas las infracciones que no están en la lista de infracciones válidas a 'otros'
df_infracciones_2['infracciones'] = df_infracciones_2['infracciones'].apply(
    lambda x: x if x in infracciones_validas else 'otros'
)

In [20]:
df_infracciones_1.to_csv('../../Datos/infracciones/df_infracciones_1.csv')
df_infracciones_2.to_csv('../../Datos/infracciones/df_infracciones_2.csv')

## Datos curiosos

In [21]:
df_infracciones_2_dirt

,anio,infracciones,total,porcentaje
0,2018,transitar sin tablilla,527,0.06
1,2018,exceso de pasajeros,4153,0.51
2,2018,niños en el asiento delantero,2601,0.32
3,2018,pasajeros en vehículos de carga,11393,1.40
4,2018,violar señales de tránsito,12044,1.48
...,...,...,...,...
642,2024,transitar sin luces,74975,3.85
643,2024,transitar con luces led,4132,0.21
644,2024,transitar con gomas lisas,8995,0.46
645,2024,a. aditamentos en las placas,2936,0.15


In [22]:
df_infracciones_2_dirt[df_infracciones_2_dirt['anio'] == 2022].sort_values(by='porcentaje', ascending=False)

,anio,infracciones,total,porcentaje
427,2022,transitar sin casco,447986,22.11
359,2022,violar la luz roja,178461,8.81
399,2022,transitar sin seguro o vencido,176513,8.71
398,2022,conducir sin licencia y/o vencida,172724,8.52
435,2022,exceso de velocidad,170879,8.43
...,...,...,...,...
419,2022,uso del freno de emergencia,60,0.00
356,2022,transportar pasajeros en vehículos fúnebres,79,0.00
392,2022,emisiones contaminantes y conversión de vehícu...,67,0.00
352,2022,capacidad de asientos,60,0.00


In [23]:
# Filtrar filas donde la columna 'infracciones' contiene la palabra 'alcohol' (sin importar mayúsculas o minúsculas)
infracciones_alcohol = df_infracciones_2_dirt[df_infracciones_2_dirt['infracciones'].str.contains('embriaguez', case=False, na=False)]

# Mostrar el resultado
infracciones_alcohol

,anio,infracciones,total,porcentaje
29,2018,manejo en estado de embriaguez,1358,0.17
134,2019,conducir en estado de embriaguez,777,0.08
231,2020,conducir en estado de embriaguez,214,0.03
332,2021,conducir en estado de embriaguez,212,0.01
433,2022,conducir en estado de embriaguez,335,0.02
534,2023,conducir en estado de embriaguez,244,0.02
635,2024,conducir en estado de embriaguez,285,0.01
